### Hello, welcome to the FMR Python Automation tutorial!

We'll go over our code and how to use it.

**Structure**

Our code is based on just a couple of Python scripts. These are:
- `instrument_base.py`
- `bop50_8d.py`
- `hp_8673g.py`
- `srs_sr830.py`
- `fmr_experiment.py`

The first four files help us initialise and interact with our instruments. The last file `fmr_experiment.py` gives us some methods to then conduct various sweeps.

**What do I need to do to use these scripts?**

You'll need to make sure that whatever Python file you're using to do your analysis (be it a simple Python script or a Jupyter `.ipynb` notebook file) has those 5 files in the same directory. Of course you'll also want to make sure that the PC that you're running these files on is also connectied with our instruments via GPIB.

For instance, we're using this file, `tutorial.ipynb` to run some example code. So we'll want to make sure that those 5 Python scripts are in the same folder as this file. Let us begin.

### Calling our Experiment Class

Our experiment has a class, which helps it gather all the resources it needs among other things. If you're unfamiliar with this, don't worry. Simply import the class like so:

In [ ]:
from fmr_experiment import Experiment

In [ ]:
# Now let's initialise the class
E = Experiment()

The cell above would show a bunch of 